## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
paths = sorted(os.listdir('./data/MIMIC'))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('PROCEDURES_ICD.csv')
paths.remove('PROCEDUREEVENTS_MV.csv')
paths.remove('MICROBIOLOGYEVENTS.csv')
paths.remove('D_CPT.csv')
paths.remove('INPUTEVENTS_CV_Filtered1.csv')
paths.remove('INPUTEVENTS_CV_Filtered2.csv')
paths.remove('INPUTEVENTS_CV_Filtered3.csv')
paths.remove('INPUTEVENTS_CV_FilteredPDA.csv')
paths.remove('OUTPUTEVENTS_Filtered.csv')
paths.remove('OUTPUTEVENTS_FilteredPDA.csv')
paths.remove('LABEVENTS_Filtered.csv')
paths.remove('LABEVENTS_FilteredPDA.csv')
paths

['ADMISSIONS.csv',
 'CHARTEVENTS.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv']

## Load Data

In [173]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv('./data/MIMIC/' + path))

  0%|                                                                                                                                                                                          | 0/15 [00:00<?, ?it/s]/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1904/2755860072.py:3: DtypeWarning: Columns (13,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv('./data/MIMIC/' + path))
 13%|███████████████████████▋                                                                                                                                                          | 2/15 [00:00<00:04,  3.01it/s]/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1904/2755860072.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv('./data/MIMIC/' + path))
 87%|█████████████████████████████████████████████████████████████████████████████████████████████

## Change Data to Time

In [175]:
for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column or 'charttime' == column or 'storetime' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:19<00:00,  1.28s/it]


# Look Data characteristics

In [177]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
D_ICD_DIAGNOSES

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,0,0,9049,9049,9049,9050,77181,NB septicemia [sepsis],Septicemia [sepsis] of newborn
1,1,1,10304,10304,10304,11403,99591,Sepsis,Sepsis
2,2,2,10305,10305,10305,11404,99592,Severe sepsis,Severe sepsis
3,3,3,13293,13293,13293,13564,67020,Puerperal sepsis-unsp,"Puerperal sepsis, unspecified as to episode of..."
4,4,4,13294,13294,13294,13565,67022,Puerprl sepsis-del w p/p,"Puerperal sepsis, delivered, with mention of p..."
5,5,5,13295,13295,13295,13566,67024,Puerperl sepsis-postpart,"Puerperal sepsis, postpartum condition or comp..."


In [178]:
i = paths.index('DIAGNOSES_ICD.csv')
DIAGNOSES_ICD = datas[i].copy()
DIAGNOSES_ICD

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,747,140,21,111970,11.0,99592
1,914,307,38,185910,3.0,99592
2,1084,477,61,189535,8.0,99591
3,1097,490,62,116009,4.0,99591
4,276,505,64,172056,3.0,99591
...,...,...,...,...,...,...
5404,648640,650341,99836,101018,8.0,99592
5405,649024,650470,99865,189346,4.0,99592
5406,649529,650720,99912,189380,8.0,99592
5407,648171,650990,99985,176670,7.0,99592


In [179]:
i = paths.index('ICUSTAYS.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,0,385,21,21,111970,216859,carevue,MICU,MICU,52,52,2135-01-30 20:53:34,2135-02-08 05:38:46,8.3647
1,1,405,41,38,185910,248910,carevue,CCU,TSICU,7,23,2166-08-10 00:29:36,2166-09-04 13:39:23,25.5485
2,2,427,63,61,189535,217135,carevue,MICU,MICU,52,52,2119-01-20 15:58:00,2119-01-22 16:11:10,2.0091
3,3,428,64,62,116009,216609,carevue,CCU,CCU,7,7,2113-02-15 00:20:44,2113-02-17 20:09:46,2.8257
4,4,430,66,64,172056,232593,carevue,MICU,MICU,52,52,2143-03-03 09:26:21,2143-03-05 17:14:25,2.3250


In [180]:
i = paths.index('PATIENTS.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,239,18,21,M,2047-04-04,2135-02-08,2135-02-08,2135-02-08,1
1,255,34,38,M,2090-08-31,NaT,NaT,NaT,0
2,275,54,61,M,2063-10-21,2119-02-03,2119-02-03,2119-02-03,1
3,276,55,62,M,2044-05-08,NaT,NaT,NaT,0
4,278,57,64,F,2116-06-27,NaT,NaT,NaT,0


In [181]:
i = paths.index('ADMISSIONS.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.drop(ADMISSIONS[ADMISSIONS['HAS_CHARTEVENTS_DATA'] == 0].index, inplace=True)
ADMISSIONS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,...,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,0,230,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,EMERGENCY ROOM ADMIT,...,Medicare,NaN,JEWISH,MARRIED,WHITE,2135-01-30 18:46:00,2135-01-30 22:05:00,SEPSIS,1,1
1,1,19,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,...,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaT,NaT,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
2,2,63,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,...,Private,NaN,CATHOLIC,MARRIED,WHITE,NaT,NaT,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,1,1
3,3,64,62,62,116009,2113-02-15 00:19:00,2113-02-19 15:30:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,...,Medicare,NaN,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,2113-02-14 19:55:00,2113-02-15 01:17:00,"SEPSIS,URINARY TRACT INFECTION",0,1
4,4,66,64,64,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,...,Medicaid,ENGL,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2143-03-03 03:46:00,2143-03-03 11:00:00,FUNGAL MENINGITIS,0,1


In [182]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,112673,111797,21,111970.0,216859.0,2135-01-31 10:30:00,30056,120.0,ml,NaN,...,328907,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,112674,111798,21,111970.0,216859.0,2135-01-31 18:15:00,30056,120.0,ml,NaN,...,2063480,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,112675,111799,21,111970.0,216859.0,2135-02-01 08:00:00,30056,120.0,ml,NaN,...,3204863,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,112676,111800,21,111970.0,216859.0,2135-02-02 21:00:00,30056,20.0,ml,NaN,...,3659956,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,112677,111801,21,111970.0,216859.0,2135-02-03 21:00:00,30056,50.0,ml,NaN,...,8121802,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [183]:
INPUTEVENTS_CV.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',
       'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE',
       'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE',
       'ORIGINALRATEUOM', 'ORIGINALSITE'],
      dtype='object')

In [184]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_MV = datas[i].copy()
INPUTEVENTS_MV.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,112673,111797,21,111970.0,216859.0,2135-01-31 10:30:00,30056,120.0,ml,NaN,...,328907,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,112674,111798,21,111970.0,216859.0,2135-01-31 18:15:00,30056,120.0,ml,NaN,...,2063480,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,112675,111799,21,111970.0,216859.0,2135-02-01 08:00:00,30056,120.0,ml,NaN,...,3204863,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,112676,111800,21,111970.0,216859.0,2135-02-02 21:00:00,30056,20.0,ml,NaN,...,3659956,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,112677,111801,21,111970.0,216859.0,2135-02-03 21:00:00,30056,50.0,ml,NaN,...,8121802,328907,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [185]:
INPUTEVENTS_MV.columns

Index(['Unnamed: 0', 'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID',
       'CHARTTIME', 'ITEMID', 'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM',
       'STORETIME', 'CGID', 'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE',
       'ORIGINALAMOUNT', 'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE',
       'ORIGINALRATEUOM', 'ORIGINALSITE'],
      dtype='object')

In [186]:
i = paths.index('OUTPUTEVENTS.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,1572072,1675124,21,111970.0,216859.0,2135-02-02 04:00:00,40055,0.0,ml,2135-02-02 04:26:00,14206,NaN,NaN,NaN
1,1572073,1675125,21,111970.0,216859.0,2135-02-02 05:00:00,40055,5.0,ml,2135-02-02 05:10:00,14206,NaN,NaN,NaN
2,1572074,1675126,21,111970.0,216859.0,2135-02-02 06:00:00,40055,3.0,ml,2135-02-02 05:46:00,14206,NaN,NaN,NaN
3,1572075,1675127,21,111970.0,216859.0,2135-02-02 08:00:00,40055,22.0,ml,2135-02-02 07:52:00,20834,NaN,NaN,NaN
4,1572076,1675128,21,111970.0,216859.0,2135-02-02 09:00:00,40055,0.0,ml,2135-02-02 09:14:00,20834,NaN,NaN,NaN


In [187]:
i = paths.index('D_ITEMS.csv')
D_ITEMS = datas[i].copy()
D_ITEMS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,290,290,27,1,% Inspir. Time,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,291,291,28,2,ABI (L),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,292,292,29,3,ABI (R),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,293,293,30,4,ABI Ankle BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,294,294,31,5,ABI Brachial BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [188]:
for i in range(len(D_ITEMS)):
    if D_ITEMS.iloc[i]['ITEMID'] == 30025:
        print(D_ITEMS.iloc[i])

Unnamed: 0.1              5782
Unnamed: 0                5782
ROW_ID                    5352
ITEMID                   30025
LABEL                  Heparin
ABBREVIATION               NaN
DBSOURCE               carevue
LINKSTO         inputevents_cv
CATEGORY                   NaN
UNITNAME                   NaN
PARAM_TYPE                 NaN
CONCEPTID                  NaN
Name: 5056, dtype: object


In [189]:
i = paths.index('D_LABITEMS.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,127,127,1,50800,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
1,128,128,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
2,129,129,3,50802,Base Excess,Blood,Blood Gas,11555-0
3,130,130,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
4,131,131,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6


In [190]:
i = paths.index('DRGCODES.csv')
DRGCODES = datas[i].copy()
DRGCODES.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,0,26583,25699,21,111970,HCFA,416,SEPTICEMIA AGE >17,NaN,NaN
1,1,26584,25700,21,111970,APR,7204,Septicemia & Disseminated Infections,4.0,4.0
2,2,25591,22272,38,185910,HCFA,541,ECMO OR TRACHEOSTOMY WITH MECHANICAL VENTILATI...,NaN,NaN
3,3,25592,22273,38,185910,APR,54,Tracheostomy W Long Term Mechanical Ventilatio...,4.0,4.0
4,4,21252,16347,61,189535,HCFA,403,LYMPHOMA & NON-ACUTE LEUKEMIA WITH COMPLICATIO...,NaN,NaN


In [191]:
i = paths.index('LABEVENTS.csv')
LABEVENTS = datas[i].copy()
LABEVENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,10085,10440,21,111970.0,51279,2135-02-03 05:04:00,2.81,2.81,m/uL,abnormal
1,10086,10441,21,111970.0,51301,2135-02-03 05:04:00,26.8,26.80,K/uL,abnormal
2,10087,10442,21,111970.0,50800,2135-02-03 05:13:00,MIX,NaN,NaN,NaN
3,10088,10443,21,111970.0,50813,2135-02-03 05:13:00,2.6,2.60,mmol/L,abnormal
4,10089,10444,21,111970.0,50817,2135-02-03 05:13:00,56,56.00,%,NaN


In [192]:
i = paths.index('PRESCRIPTIONS.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,435,1390475,21,111970,216859.0,2135-01-31,2135-01-31,BASE,NS (Mini Bag Plus),NaN,NaN,NS/MBP100I,001210,338055318.0,100mL Bag,100,ml,1,BAG,IV
1,436,1389751,21,111970,216859.0,2135-01-31,2135-01-31,MAIN,Levothyroxine Sodium,Levothyroxine Sodium,Levothyroxine Sodium,LEVO50,006649,74455211.0,50mcg Tablet,50,mcg,1,TAB,PO
2,437,1379358,21,111970,216859.0,2135-01-31,2135-01-31,MAIN,Meropenem,NaN,NaN,MERO500I,026488,310032520.0,500mg Vial,500,mg,1,VIAL,IV
3,438,1390474,21,111970,216859.0,2135-01-31,2135-02-01,BASE,SW,NaN,NaN,KCLBASE2,NaN,0.0,50ml Bag,100,ml,100,ml,IV
4,439,1390472,21,111970,216859.0,2135-01-31,2135-02-01,BASE,Iso-Osmotic Dextrose,NaN,NaN,VANCOBASE,NaN,0.0,200ml Bag,200,ml,200,ml,IV


In [193]:
i = paths.index('CHARTEVENTS.csv')
CHARTEVENTS = datas[i].copy()
CHARTEVENTS.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,248563,248563,248563,86199586,10114,167957,234989.0,1,2171-10-30 20:30:00,2171-10-30 20:33:00,14700,33%,NaN,NaN,NaN,NaN,NaN,NotStopd
1,248376,248376,248376,86199399,10114,167957,234989.0,1,2171-10-30 18:00:00,2171-10-30 18:21:00,14700,33%,NaN,NaN,NaN,NaN,NaN,NotStopd
2,250905,250905,250905,86201928,10114,167957,234989.0,1,2171-11-04 10:00:00,2171-11-04 10:00:00,21570,NaN,NaN,NaN,NaN,NaN,NaN,D/C'd
3,341403,341403,341403,86336049,10126,160445,249805.0,24,2171-08-16 05:00:00,2171-08-16 04:44:00,16866,2cc,NaN,NaN,NaN,NaN,NaN,NotStopd
4,114642,114642,114642,85821483,10019,177759,228977.0,24,2163-05-15 05:00:00,2163-05-15 04:40:00,20088,1.4,1.4,NaN,NaN,NaN,NaN,NotStopd


# Features

In [21]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Load dataset after split time

In [218]:
path = './data/dataset_split_4_hour.csv'
dataset = pd.read_csv(path)

# change time column data's type to timestamp type
dataset['INTIME'] = dataset['INTIME'].apply(lambda x : pd.Timestamp(x))
dataset['OUTTIME'] = dataset['OUTTIME'].apply(lambda x : pd.Timestamp(x))
dataset['DEATHTIME'] = dataset['DEATHTIME'].apply(lambda x : pd.Timestamp(x))
dataset.columns = ['Unnamed:0', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'Gender',
                   'Age', 'DEATHTIME', 're_admission']
dataset = dataset.iloc[:, 1:]
dataset.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,Gender,Age,DEATHTIME,re_admission
0,21,111970,216859,2135-01-30 20:53:34,2135-01-31 00:53:34,M,87.882192,2135-02-08 02:08:00,0
1,21,111970,216859,2135-01-31 00:53:34,2135-01-31 04:53:34,M,87.882192,2135-02-08 02:08:00,0
2,21,111970,216859,2135-01-31 04:53:34,2135-01-31 08:53:34,M,87.882192,2135-02-08 02:08:00,0
3,21,111970,216859,2135-01-31 08:53:34,2135-01-31 12:53:34,M,87.882192,2135-02-08 02:08:00,0
4,21,111970,216859,2135-01-31 12:53:34,2135-01-31 16:53:34,M,87.882192,2135-02-08 02:08:00,0


## Load variable range

In [220]:
var_range = pd.read_csv('variable_range.csv')
var_range.index = var_range.iloc[:, 0]
var_range = var_range.iloc[:, 1:]
var_range['GROUP_ID'] = var_range['GROUP_ID'].apply(lambda x : [int(s) for s in x.split(',')] if type(x) == type('') else x)
var_range.head()

,OUTLIER LOW,VALID LOW,IMPUTE,VALID HIGH,OUTLIER_HIGH,GROUP_ID
Unnamed: 0,,,,,,
Albumin,0.0,0.6,3.1,6.0,60.0,"[50862, 772, 1521, 227456]"
Arterial pH,NaN,NaN,NaN,NaN,NaN,"[780, 1126, 223830, 50820]"
Calcium,NaN,NaN,NaN,NaN,NaN,"[225625, 50893, 786, 1522]"
Glucose,0.0,33.0,128.0,2000.0,2200.0,"[220621, 225664, 50809, 811, 807, 50931, 22653..."
Hemoglobin,0.0,0.0,10.2,25.0,30.0,"[220228, 51222, 50811, 814]"


In [221]:
var_range.index

Index(['Albumin', 'Arterial pH', 'Calcium', 'Glucose', 'Hemoglobin',
       'Magnesium', 'PTT', 'Postassium', 'SGPT', 'Arterial blood gas', 'BUN',
       'HCO3', 'INR', 'Arterial Lactate', 'CO2', 'Creatinine',
       'Ionized Calcium', 'PT', 'Platelets Count', 'SGOT', 'Total Bilirubin',
       'White Blood Cell Count', 'Diastolic Blood Pressure',
       'Systolic Blood Pressure', 'Mean Blood Pressure', 'PaCO2', 'PaO2',
       'FiO2', 'PaO/FiO2 ratio', 'Respiratory rate', 'Temperature (Celsius)',
       'Weight (kg)', 'Heart Rate', 'SpO2', 'Mechanical Ventilation'],
      dtype='object', name='Unnamed: 0')

## Merge features to dataset

In [ ]:
for feature in tqdm(var_range.index):
    # count records in same period
    dataset['records'] = pd.Series([0] * len(dataset), dtype = np.int64)
    
    # add new feature column
    dataset[feature] = pd.Series([np.nan] * len(dataset), dtype = np.float64)

    # find all feature recoords in LABEVENTS
    labevent = LABEVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in LABEVENTS['ITEMID']]]

    for i in range(len(labevent)):
        subject_id = labevent.iloc[i]['SUBJECT_ID']
        hadm_id = labevent.iloc[i]['HADM_ID']
        time = labevent.iloc[i]['CHARTTIME']
        value = labevent.iloc[i]['VALUENUM']
        index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id]).index
        for idx in index:
            if time - dataset.iloc[idx]['STARTTIME'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['ENDTIME'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1

    
    # find all feature recoords in CHARTEVENTS
    chartevent = CHARTEVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in CHARTEVENTS['itemid']]]

    for i in range(len(chartevent)):
        subject_id = chartevent.iloc[i]['subject_id']
        hadm_id = chartevent.iloc[i]['hadm_id']
        icustay_id = chartevent.iloc[i]['icustay_id']
        time = chartevent.iloc[i]['charttime']
        value = chartevent.iloc[i]['valuenum']
        index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id][dataset['ICUSTAY_ID'] == icustay_id]).index
        for idx in index:
            if time - dataset.iloc[idx]['STARTTIME'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['ENDTIME'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1

    
    # find all feature recoords in OUTPUTEVENTS
    outputevent = OUTPUTEVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in OUTPUTEVENTS['ITEMID']]]

    for i in range(len(outputevent)):
        subject_id = outputevent.iloc[i]['subject_id']
        hadm_id = outputevent.iloc[i]['hadm_id']
        icustay_id = outputevent.iloc[i]['icustay_id']
        time = outputevent.iloc[i]['charttime']
        value = outputevent.iloc[i]['valuenum']
        index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id][dataset['ICUSTAY_ID'] == icustay_id]).index
        for idx in index:
            if time - dataset.iloc[idx]['STARTTIME'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['ENDTIME'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1

  0%|                                                                                                                                                                                          | 0/35 [00:00<?, ?it/s]/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1904/41043093.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id]).index
/var/folders/r3/tfk14xq149s3ncn1ws04wscr0000gn/T/ipykernel_1904/41043093.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id][dataset['ICUSTAY_ID'] == icustay_id]).index
  3%|█████                                                                                                                                                                          | 1/35 [07:40<4:20:47, 460.22s/it]/var/folders/r3/tfk14xq149s3ncn1ws0

In [212]:
dataset

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,Gender,Age,DEATHTIME,re_admission,Albumin,records
2,21,111970,216859,2135-01-31 04:53:34,2135-01-31 08:53:34,M,87.882192,2135-02-08 02:08:00,0,2.1,1
25,21,111970,216859,2135-02-04 00:53:34,2135-02-04 04:53:34,M,87.882192,2135-02-08 02:08:00,0,1.7,1
62,38,185910,248910,2166-08-12 00:29:36,2166-08-12 04:29:36,M,75.991781,NaT,0,2.2,1
80,38,185910,248910,2166-08-15 00:29:36,2166-08-15 04:29:36,M,75.991781,NaT,0,1.5,1
94,38,185910,248910,2166-08-17 08:29:36,2166-08-17 12:29:36,M,75.991781,NaT,0,1.7,1
...,...,...,...,...,...,...,...,...,...,...,...
328471,99836,101018,200347,2116-06-19 23:09:08,2116-06-20 03:09:08,F,52.178082,NaT,0,2.8,1
328482,99836,101018,240303,2116-06-26 02:28:30,2116-06-26 06:28:30,F,52.232877,NaT,1,2.6,1
328487,99836,101018,240303,2116-06-26 22:28:30,2116-06-27 02:28:30,F,52.232877,NaT,1,2.4,1
328512,99836,101018,240303,2116-07-01 02:28:30,2116-07-01 06:28:30,F,52.232877,NaT,1,2.5,1


In [203]:
dataset

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,Gender,Age,DEATHTIME,re_admission,Albumin
0,21,111970,216859,2135-01-30 20:53:34,2135-01-31 00:53:34,M,87.882192,2135-02-08 02:08:00,0,NaN
1,21,111970,216859,2135-01-31 00:53:34,2135-01-31 04:53:34,M,87.882192,2135-02-08 02:08:00,0,NaN
2,21,111970,216859,2135-01-31 04:53:34,2135-01-31 08:53:34,M,87.882192,2135-02-08 02:08:00,0,2.1
3,21,111970,216859,2135-01-31 08:53:34,2135-01-31 12:53:34,M,87.882192,2135-02-08 02:08:00,0,NaN
4,21,111970,216859,2135-01-31 12:53:34,2135-01-31 16:53:34,M,87.882192,2135-02-08 02:08:00,0,NaN
...,...,...,...,...,...,...,...,...,...,...
328600,99865,189346,259297,2184-12-30 07:14:22,2184-12-30 11:14:22,F,77.569863,2184-12-30 00:01:00,0,NaN
328601,99865,189346,259297,2184-12-30 11:14:22,2184-12-30 15:14:22,F,77.569863,2184-12-30 00:01:00,0,NaN
328602,99865,189346,259297,2184-12-30 15:14:22,2184-12-30 19:14:22,F,77.569863,2184-12-30 00:01:00,0,NaN
328603,99865,189346,259297,2184-12-30 19:14:22,2184-12-30 23:14:22,F,77.569863,2184-12-30 00:01:00,0,1.5
